In [1]:
!pip install kaggle
!mkdir -p ~/.kaggle
!mv kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json
!kaggle datasets download -d apollo2506/eurosat-dataset

eurosat-dataset.zip: Skipping, found more recently modified local copy (use --force to force download)


In [3]:
!unzip eurosat-dataset.zip

Archive:  eurosat-dataset.zip
replace EuroSAT/AnnualCrop/AnnualCrop_1.jpg? [y]es, [n]o, [A]ll, [N]one, [r]ename: 

In [4]:
import pandas as pd
import numpy as np
import cv2
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC
from sklearn.metrics import classification_report, accuracy_score
import json
import os

In [5]:
train_df = pd.read_csv('EuroSAT/train.csv')

In [6]:
with open('/content/EuroSAT/label_map.json', 'r') as f:
    label_map = json.load(f)

In [7]:
def load_image(image_path):
    if os.path.exists(image_path):
        img = cv2.imread(image_path)
        if img is not None:  # Check if the image is loaded successfully
            img = cv2.resize(img, (256,256))  # Resize to 64x64
            img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)  # Convert to RGB
            return img
    print("Error loading image:", image_path)
    return None

In [8]:
train_df_subset = train_df.sample(n=1000, random_state=42)

In [9]:
image_directory = 'EuroSAT'

X = []
y = []
for idx, row in train_df_subset.iterrows():
    img_path = os.path.join(image_directory, row['Filename'])  # Construct full image path
    img = load_image(img_path)
    if img is not None:
        X.append(img.flatten())  # Flatten image to use as feature
        y.append(label_map[row['ClassName']])



In [10]:
X = np.array(X)
y = np.array(y)

In [11]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [12]:
svm_model = SVC()
svm_model.fit(X_train, y_train)

SVC()

In [13]:
y_pred = svm_model.predict(X_test)


In [14]:
print("Accuracy:", accuracy_score(y_test, y_pred))


Accuracy: 0.56


In [15]:
print("Classification Report:\n", classification_report(y_test, y_pred))

Classification Report:
               precision    recall  f1-score   support

           0       0.35      0.57      0.43        23
           1       0.64      1.00      0.78        25
           2       0.50      0.48      0.49        25
           3       1.00      0.07      0.12        15
           4       0.87      0.90      0.89        30
           5       0.88      0.47      0.61        15
           6       0.00      0.00      0.00        17
           7       0.32      0.60      0.42        15
           8       0.32      0.46      0.37        13
           9       0.92      0.55      0.69        22

    accuracy                           0.56       200
   macro avg       0.58      0.51      0.48       200
weighted avg       0.60      0.56      0.53       200



/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
